This is an implementation of partial policy iteration with Q-networks.
The algorithm is run on path selection task for symbolic execution.
Each epoch we communicate with jar-file for data gathering and wandb for logging.

### Imports, meta

In [ ]:
# %%capture

from IPython.display import Javascript
def resize_colab_cell():
  display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 450})'))
get_ipython().events.register('pre_run_cell', resize_colab_cell)

import numpy as np
from numpy import random
import copy
import inspect
import torch
from torch import nn
import torch.onnx
import json
from tqdm import tqdm, trange
from time import time
import os
import sklearn
from sklearn import tree
import math

# !pip install wandb
import wandb

# !pip install onnx==1.12
# import onnx

# !pip install onnxruntime
# import onnxruntime

with open('../Game_env/jar_config.txt', 'w') as jar_config:
    jar_config.write(json.dumps({"algorithm": "TD"}))

### Args (potentially immutable), login

In [ ]:
# %%capture
wandb.login()

<IPython.core.display.Javascript object>

wandb: Currently logged in as: andrey_podivilov. Use `wandb login --relogin` to force relogin


True

In [ ]:
batch_size = 1024
device = 'cuda' if torch.cuda.is_available() else 'cpu'
td_gamma=0.99
json_path = '../Data/current_dataset.json' # '/content/branching_small.json'
use_TD = True
use_MC = 1 - use_TD
use_fork_discount = False

jar_command = '/home/st-andrey-podivilov/java16/usr/lib/jvm/bellsoft-java16-amd64/bin/java -jar ../Game_env/usvm-jvm/build/libs/usvm-jvm-new.jar ../Game_env/jar_config.txt > ../Game_env/jar_log.txt'

device

<IPython.core.display.Javascript object>

'cuda'

### Models, modules

In [ ]:
class FFM_layer(torch.nn.Module):
    """
    Why Not?
    """
    def __init__(self, input_dim):
      super().__init__()
      assert input_dim%2 == 0, 'even input_dim is more convenient'
      self.fourier_matrix = torch.nn.Linear(input_dim, int(input_dim), bias=False)
      nn.init.normal_(
          self.fourier_matrix.weight,
          std=1/np.sqrt(input_dim),
      )
      self.fourier_matrix.weight.requires_grad_(False)

    def forward(self, x):
      pre = x # self.fourier_matrix(x)
      s = torch.sin(pre)
      c = torch.cos(pre)
      return torch.cat([x,s,c], dim=-1)

def get_mlp_setup(use_FFM=False):
    mlp = nn.Sequential(
        nn.LazyLinear(512),
        nn.ReLU(),
        nn.Linear(512,256),
        nn.LayerNorm(256),
        FFM_layer(256) if use_FFM else nn.Identity(),
        nn.LazyLinear(512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,1),
    ).to(device)
    mlp_opt = torch.optim.AdamW(mlp.parameters(), lr=3e-4, weight_decay=0.1, betas=(0.9, 0.99))
    return mlp, mlp_opt

# to check features' strength
r_tree = tree.DecisionTreeRegressor(max_depth=1000, )

<IPython.core.display.Javascript object>

### Data

In [ ]:
class Trajectories:
  """
  Contains all kinds of data in a form of tensor.
  train_tensors are raw and derivative features of visited states.
  Action and state embeddings are effectively the same, fyi.
  """
  def __init__(self,
               path=json_path,
               td_gamma=td_gamma,
               eval_condition = (lambda x: x%hash_modulo == 0),
              ):
    self.eval_condition = eval_condition
    self.td_gamma = td_gamma
    self.j_file = json.load(open(path))
    self.feature_names = self.j_file['scheme'][0]
    self.feature_names2ids = {self.feature_names[i]:i for i in range(len(self.feature_names))}
    self.train_tensors = self.j2torch(self.j_file) #sarsa list of 5 tensors of length n_states-n_trajectories f, f_n, r, R, is_last
    self.n_sarsa_pairs = self.n_sarsa_pairs()

  def j2torch(self, j_file):
    """
    transforms json to data tensors
    """
    features, features_next, rewards, Returns, is_last = [], [], [], [], []
    chosenStId_idx = self.j_file['scheme'].index('chosenStateId')
    rewards_idx = self.j_file['scheme'].index('reward')

    for tr in self.j_file['paths']:
      if self.eval_condition(tr[0]):
        continue
      tr = tr[1]
      tr_rewards = [tr[i][rewards_idx] for i in range(len(tr))]
      rewards += tr_rewards
      do_discount = torch.Tensor([1]*len(tr))
      if use_fork_discount:
        is_cfg_fork_idx = self.j_file['scheme'].index('is_cfg_fork')
        do_discount = [tr[i][is_cfg_fork_idx] for i in range(len(tr))]
      tr_Returns = self.tr_rewards_to_returns(tr_rewards, do_discount)
      Returns += tr_Returns

      tr_features = [tr[i][0][tr[i][chosenStId_idx]] for i in range(len(tr))]
      is_last += [0]*(len(tr_features)-1) + [1]
      features += tr_features
      features_next += tr_features[1:] + [[0]*len(tr_features[0])]
    rewards = torch.Tensor(rewards).to(device)
    features = torch.Tensor(features).to(device)
    features_next = torch.Tensor(features_next).to(device)
    Returns = torch.Tensor(Returns).to(device)
    is_last = torch.Tensor(is_last).to(device)
    return [features, features_next, rewards, Returns, is_last]

  def tr_rewards_to_returns(self, tr_rewards, do_discount):
    tr_R = [0]*(len(tr_rewards)-1) + [tr_rewards[-1]]
    for i in range(len(tr_rewards)-2, -1, -1):
        tr_R[i] = tr_rewards[i] + (self.td_gamma**do_discount[i]) * tr_R[i+1]
    return tr_R

  def n_sarsa_pairs(self):
     return len(self.train_tensors[0])

  def sample_batch(self, n=batch_size):
    ids = torch.tensor(random.choice(self.n_sarsa_pairs, size=n)).long()
    sampled = [t[ids] for t in self.train_tensors]
    return sampled

  def update_data_on_path(self, path, model):
    """
    Communication with jar file on a server.
    """
    x = torch.randn(1, self.train_tensors[0][0].shape[0], requires_grad=True).to(device)
    torch_model = model.eval()
    torch_out = torch_model(x)

    torch.onnx.export(torch_model,
                      x,
                      '../Game_env/model.onnx',
                      opset_version=13,
                      export_params=True,
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'],
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'},
                                    },
                      )

    os.system(jar_command)

  def evaluate_data(self,
           factors=torch.Tensor([1, 0.99, 0.95]),
           eval_condition = None,
           verbose=True,
           wandb_prefix = 'val',
           ):
    if eval_condition is None:
        eval_condition = self.eval_condition
    rewards_idx = self.j_file['scheme'].index('reward')
    for f in factors:
      size = 0
      tr_lengths = []
      trs_R = []
      for tr in self.j_file['paths']:
        if not eval_condition(tr[0]):
          continue
        tr=tr[1]
        size += len(tr)
        tr_lengths += [len(tr)]
        tr_rewards = [tr[i][rewards_idx] for i in range(len(tr))]
        trs_R += [0]
        do_discount = torch.Tensor([1]*len(tr))
        if use_fork_discount:
          is_cfg_fork_idx = self.j_file['scheme'].index('is_cfg_fork')
          do_discount = [tr[i][is_cfg_fork_idx] for i in range(len(tr))]
        for i in range(len(tr_rewards)-1, -1, -1):
          trs_R[-1] = tr_rewards[i] + (f ** do_discount[i]) * trs_R[-1]
      log = {}
      log[f'{wandb_prefix} size'] = size
      log[f'{wandb_prefix}_eval/mean {f:.2f} discount '] = torch.Tensor(trs_R).mean()
      log[f'{wandb_prefix}_eval/median {f:.2f} discount '] = torch.Tensor(trs_R).median()
      # log[f'{wandb_prefix}_eval/95_quanile {f:.2f} discount'] = torch.Tensor(trs_R).quantile(q=0.95)
      log[f'{wandb_prefix} Return by trjs {f:.2f} hist (previous)'] = wandb.Histogram(np_histogram=np.histogram(trs_R, bins=50, ))
#       log[f'{wandb_prefix} lengths hist'] = wandb.Histogram(np_histogram=np.histogram(tr_lengths, bins=30, ))
      if verbose:
          wandb.log(log.copy())
      log['Returns'] = trs_R
      log[f'{wandb_prefix} lengths'] = tr_lengths
    return log

<IPython.core.display.Javascript object>

### Logger

In [ ]:
class Logger:
  """
  Supporting class, to be expanded.
  Stores logging methods and relevant data.
  """
  def __init__(
      self,
      NN_setup,
      batch_size=batch_size,
      between_logs = 0,
  ):
    self.model = NN_setup['model']
    self.optimizer = NN_setup['optimizer']
    self.grad = None
    self.weight = None
    self.running_grad_mean = torch.zeros(len([p for p in self.model.parameters() if p.requires_grad])).to(device)
    self.running_grad2_mean = torch.zeros(len([p for p in self.model.parameters() if p.requires_grad])).to(device)
    self.log_gamma = torch.tensor(0.95).to(device)
    self.between_logs = between_logs

  @torch.no_grad()
  def link_model(self,):
    self.grad = [p.grad.detach() for p in self.model.parameters() if p.requires_grad]
    self.weight = [p.detach() for p in self.model.parameters()]

  @torch.no_grad()
  def list_norm(self, l, p=2):
    n = 0
    for t in l:
      n += t.detach().norm(p) ** p
    return n.item() ** (1/p)

  @torch.no_grad()
  def list_cos_dist(self, a, b):
    a_norm = self.list_norm(a, 2)
    b_norm = self.list_norm(b, 2)
    product = sum([torch.dot(torch.flatten(a[i]), torch.flatten(b[i])).item() for i in range(len(a))])
    return product/(a_norm*b_norm)

  @torch.no_grad()
  def on_list(self, a, b, operation):
    assert len(a) == len(b), 'lists lengths differ'
    return [operation(a[i], b[i]) for i in range(len(a))]

  @torch.no_grad()
  def running_mean(self, a, b):
    return [a[i].mul(self.log_gamma) + b[i].mul(1 - self.log_gamma) for i in range(len(a))]

  @torch.no_grad()
  def step(self):
    self.running_grad_mean = self.running_mean(self.running_grad_mean, self.grad)
    self.running_grad2_mean = self.running_mean(self.running_grad2_mean, [g**2 for g in self.grad])

  @torch.no_grad()
  def grad_stdev(self):
    dev = [self.running_grad2_mean[i] - m**2 for i, m in enumerate(self.running_grad_mean)]
    return [torch.sqrt(torch.maximum(torch.tensor(0), d)) for d in dev]

<IPython.core.display.Javascript object>

### Trainer


In [ ]:
class NN_Trainer:
  def __init__(
      self,
      NN_setup,
      logger=None,
      trajectories=None,
      batch_size=batch_size,
      n_batches=1000,
      target_update_steps = 20,
      td_gamma=td_gamma,
      ):
    self.n_batches = n_batches
    self.batch_number = -1
    self.td_gamma = td_gamma
    self.model = NN_setup['model'].train()
    self.target_model = copy.deepcopy(self.model).eval()
    self.optimizer = NN_setup['optimizer']
    self.trajectories = trajectories
    self.batch_size = batch_size
    self.target_update_steps = target_update_steps
    self.logger = logger
    self.log = {}

  def MC_loss(self,
              features,
              Returns,
              ):
    """
    Optional loss computed from Monte Carlo estimates
    """
    q = self.model(features)
    loss = torch.mean((q.squeeze() - Returns)**2)
    if self.batch_number % (self.logger.between_logs+1) == 0:
      self.log['Q-function mean'] = torch.mean(q.detach()).item()
      self.log['Q-function stdev'] = torch.std(q.detach()).item()
      hist = wandb.Histogram(np_histogram=np.histogram(q.detach().to('cpu'), bins=30, ))
      self.log['Q-function hist'] = hist
      hist = wandb.Histogram(np_histogram=np.histogram(Returns.detach().to('cpu'), bins=30, ))
      self.log['MC Return estimate hist'] = hist
      self.log['Return mean'] = Returns.mean().item()
      self.log['Return std'] = Returns.std().item()
    return torch.sqrt(loss)

  def td1_loss(self,
               features,
               features_next,
               rewards,
               is_last,
    ):
    """
    Temporal difference loss
    """
    q = self.model(features)
    with torch.no_grad():
      q_next = self.target_model(features_next).detach()
    TD = q.squeeze() - (rewards + q_next.squeeze() * self.td_gamma * (1-is_last))
    loss = torch.mean(TD**2)

    if self.batch_number % (self.logger.between_logs+1) == 0:
      self.log['Q-function mean'] = torch.mean(q.detach()).item()
      self.log['Q-function stdev'] = torch.std(q.detach()).item()
      hist = wandb.Histogram(np_histogram=np.histogram(q.detach().to('cpu'), bins=30, ))
      self.log['Q-function hist'] = hist
      hist = wandb.Histogram(np_histogram=np.histogram(TD.detach().to('cpu'), bins=80, ))
      self.log['TD hist'] = hist
    return loss /10

  def learn_q(self, ):
    """
    Approximates Q-function of previous policy by iterating over collected data.
    Q-function is used later on to update data gathering policy in an argmax fasion.
    """
    logger=self.logger
    for i in trange(self.n_batches):
      self.batch_number = i
      if self.batch_number % self.target_update_steps == 0:
        self.target_model = copy.deepcopy(self.model).eval()
      f, f_next, r, Returns, is_last = self.trajectories.sample_batch(self.batch_size)
      if use_TD:
        loss = self.td1_loss(f, f_next, r, is_last)
      if use_MC:
        loss = self.MC_loss(f, Returns)
      self.optimizer.zero_grad()
      loss.backward()
      torch.nn.utils.clip_grad_norm_(self.model.parameters(), 20)
      self.optimizer.step()

      logger.link_model()
      logger.step()

      if self.batch_number % (logger.between_logs+1) == 0:
        self.log.update({
            # 'batch_number': self.batch_number,
            'loss': loss.item(),
            'reward mean': torch.mean(r).item(),
            'grad L2': logger.list_norm(logger.grad, 2),
            'weight L2': logger.list_norm(logger.weight, 2),
            # 'grad_stdev L2': logger.list_norm(logger.grad_stdev(), 2),
            # 'some_feature_mean': features[:, self.trajectories.feature_names2ids['some_feature']].mean()
        })
        wandb.log(self.log)
        self.log = {}

<IPython.core.display.Javascript object>

### Procedure

In [ ]:
# We start with "BFS" heuristic as policy (not to be confused with naive BFS)
time_before = time()
os.system('rm -f ../Game_env/model.onnx')
os.system(jar_command)
print('initial data gathering time:', time() - time_before)

epochs = 30
eval_conditions=[(lambda x: x%5==0)]
td_gammas = [0.99]

for (eval_condition, td_gamma) in zip(eval_conditions, td_gammas):
    mlp, mlp_opt = get_mlp_setup(use_FFM=True)
    run = wandb.init(
          project="PS TD",
          name=f'TD, val {inspect.getsourcelines(eval_condition)[0][0].split("x")[-1].split(")")[0]}',
          config={
              'algorithm': 'partial policy iteration q-function',
              'model': 'mlp',
          }
    )


    for epoch in range(epochs):
        trajectories = Trajectories(json_path,
                                    eval_condition=eval_condition,
                                    td_gamma=td_gamma,
                                   )
        logger = Logger(NN_setup={'model': mlp, 'optimizer': mlp_opt},
                        batch_size=batch_size,
                        between_logs = 10,
                       )
        trainer = NN_Trainer(
          {'model': mlp, 'optimizer': mlp_opt},
          logger=logger,
          trajectories=trajectories,
          batch_size=batch_size,
          n_batches=500,
          td_gamma=td_gamma,
          )

        trajectories.evaluate_data()
        trajectories.evaluate_data(wandb_prefix='train',
                                 eval_condition=(lambda x: not trajectories.eval_condition(x)),
                                 )
        trainer.learn_q()

        wandb.log({'epoch': epoch,
                })

        time_before = time()
        trajectories.update_data_on_path(path='../Data/current_dataset.json', model=trainer.model)
        print('Data gathering time: ', time()-time_before)

    trajectories.update_data_on_path(path='../Data/current_dataset.json', model=trainer.model)
    trajectories.evaluate_data()

    checkpoint = {
    'model': mlp,
    }
    # torch.save(checkpoint, os.path.join(wandb.run.dir, f'mlp for TD multistep'))
    wandb.finish()

<IPython.core.display.Javascript object>

246 [main] INFO org.jooq.aR - 
                                      
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@  @@        @@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@        @@@@@@@@@@
@@@@@@@@@@@@@@@@  @@  @@    @@@@@@@@@@
@@@@@@@@@@  @@@@  @@  @@    @@@@@@@@@@
@@@@@@@@@@        @@        @@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@        @@        @@@@@@@@@@
@@@@@@@@@@    @@  @@  @@@@  @@@@@@@@@@
@@@@@@@@@@    @@  @@  @@@@  @@@@@@@@@@
@@@@@@@@@@        @@  @  @  @@@@@@@@@@
@@@@@@@@@@        @@        @@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@  @@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@  Thank you for using jOOQ 3.14.16
                                      


initial data gathering time: 200.15660095214844


/home/st-andrey-podivilov/.conda/envs/env1/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:02<00:00, 188.89it/s]


...

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



249 [main] INFO org.jooq.aR - 
                                      
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@  @@        @@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@        @@@@@@@@@@
@@@@@@@@@@@@@@@@  @@  @@    @@@@@@@@@@
@@@@@@@@@@  @@@@  @@  @@    @@@@@@@@@@
@@@@@@@@@@        @@        @@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@        @@        @@@@@@@@@@
@@@@@@@@@@    @@  @@  @@@@  @@@@@@@@@@
@@@@@@@@@@    @@  @@  @@@@  @@@@@@@@@@
@@@@@@@@@@        @@  @  @  @@@@@@@@@@
@@@@@@@@@@        @@        @@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@  @@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@  Thank you for using jOOQ 3.14.16
                                      
Aborted (core dumped)


Data gathering time:  128.94689965248108


 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌  | 491/500 [00:01<00:00, 277.61it/s]


KeyboardInterrupt: 

In [ ]:
# exit()